Importamos las librerias a utilizar

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
taxis = pd.read_parquet('../Data/Normalized/taxis_2023.parquet')
taxis.head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount,Servicio,Pickup_borough
0,2023-01-01 00:26:10,2023-01-01 00:37:11,1.0,2.58,24.18,Green,Manhattan
1,2023-01-01 00:51:03,2023-01-01 00:57:49,1.0,1.81,15.84,Green,Manhattan
2,2023-01-01 00:35:12,2023-01-01 00:41:32,1.0,0.00,11.64,Green,Queens
3,2023-01-01 00:13:14,2023-01-01 00:19:03,1.0,1.30,10.20,Green,Manhattan
4,2023-01-01 00:33:04,2023-01-01 00:39:02,1.0,1.10,8.00,Green,Manhattan


VAMOS A REALIZAR ALGUNAS TRANSFORMACIONES 

Voy a separar lo que es las fechas de los horario, lo cual asi luego podamos tener presicion en los resultados

In [3]:
taxis['fecha_inicio'] = taxis['tpep_pickup_datetime'].dt.date
taxis['horario_inicio'] = taxis['tpep_pickup_datetime'].dt.time
taxis['fecha_final'] = taxis['tpep_dropoff_datetime'].dt.date
taxis['horario_final'] = taxis['tpep_dropoff_datetime'].dt.time

print(taxis[['tpep_pickup_datetime', 'fecha_inicio', 'horario_inicio', 'tpep_dropoff_datetime', 'fecha_final', 'horario_final']].head())

  tpep_pickup_datetime fecha_inicio horario_inicio tpep_dropoff_datetime  \
0  2023-01-01 00:26:10   2023-01-01       00:26:10   2023-01-01 00:37:11   
1  2023-01-01 00:51:03   2023-01-01       00:51:03   2023-01-01 00:57:49   
2  2023-01-01 00:35:12   2023-01-01       00:35:12   2023-01-01 00:41:32   
3  2023-01-01 00:13:14   2023-01-01       00:13:14   2023-01-01 00:19:03   
4  2023-01-01 00:33:04   2023-01-01       00:33:04   2023-01-01 00:39:02   

  fecha_final horario_final  
0  2023-01-01      00:37:11  
1  2023-01-01      00:57:49  
2  2023-01-01      00:41:32  
3  2023-01-01      00:19:03  
4  2023-01-01      00:39:02  


Procedemos a eliminar las columnas de fechas anteriores

In [4]:
taxis = taxis.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [5]:
taxis

,passenger_count,trip_distance,total_amount,Servicio,Pickup_borough,fecha_inicio,horario_inicio,fecha_final,horario_final
0,1.0,2.58,24.18,Green,Manhattan,2023-01-01,00:26:10,2023-01-01,00:37:11
1,1.0,1.81,15.84,Green,Manhattan,2023-01-01,00:51:03,2023-01-01,00:57:49
2,1.0,0.00,11.64,Green,Queens,2023-01-01,00:35:12,2023-01-01,00:41:32
3,1.0,1.30,10.20,Green,Manhattan,2023-01-01,00:13:14,2023-01-01,00:19:03
4,1.0,1.10,8.00,Green,Manhattan,2023-01-01,00:33:04,2023-01-01,00:39:02
...,...,...,...,...,...,...,...,...,...
1846358,3.0,2.36,24.72,Yellow,Manhattan,2023-12-21,21:44:02,2023-12-21,21:57:59
1846359,1.0,0.60,10.90,Yellow,Manhattan,2023-12-13,21:40:15,2023-12-13,21:42:47
1846360,1.0,3.50,36.60,Yellow,Manhattan,2023-12-12,18:09:08,2023-12-12,18:33:53
1846361,2.0,1.12,20.00,Yellow,Manhattan,2023-12-14,17:33:01,2023-12-14,17:48:33


Vamos a proceder a visualizar columnas por columnas y vamos a eliminar valores que no sean relevantes para el modelo

In [6]:
taxis['passenger_count'].unique()

array([ 1.,  2.,  4.,  3.,  6.,  5.,  7.,  8.,  9., nan,  0.])

In [7]:
taxis = taxis.dropna(subset=['passenger_count'])

taxis = taxis.loc[taxis['passenger_count'] != 0]

taxis['passenger_count'] = taxis['passenger_count'].astype(int)

print("Valores únicos después de la limpieza:", taxis['passenger_count'].unique())

Valores únicos después de la limpieza: [1 2 4 3 6 5 7 8 9]


In [8]:
taxis.head(2)

,passenger_count,trip_distance,total_amount,Servicio,Pickup_borough,fecha_inicio,horario_inicio,fecha_final,horario_final
0,1,2.58,24.18,Green,Manhattan,2023-01-01,00:26:10,2023-01-01,00:37:11
1,1,1.81,15.84,Green,Manhattan,2023-01-01,00:51:03,2023-01-01,00:57:49


In [9]:
taxis['trip_distance'].unique()

array([2.580000e+00, 1.810000e+00, 0.000000e+00, ..., 4.927000e+01,
       1.075447e+04, 3.090000e+01])

In [10]:
print(taxis['trip_distance'].describe())

count    1.790351e+06
mean     3.282464e+00
std      3.860948e+01
min      0.000000e+00
25%      1.090000e+00
50%      1.800000e+00
75%      3.310000e+00
max      2.915188e+04
Name: trip_distance, dtype: float64


In [11]:
taxis.drop(taxis[taxis['trip_distance'] > 60].index, inplace=True)

In [12]:
taxis['trip_distance'].unique()

array([ 2.58,  1.81,  0.  , ..., 36.88, 49.27, 30.9 ])

In [13]:
taxis['Servicio'].unique()

array(['Green', 'Yellow'], dtype=object)

In [14]:
taxis['Pickup_borough'].unique()

array(['Manhattan', 'Queens', 'Brooklyn', 'Staten Island', 'EWR'],
      dtype=object)

In [15]:
taxis['fecha_inicio'].unique()

array([datetime.date(2023, 1, 1), datetime.date(2022, 12, 9),
       datetime.date(2023, 1, 2), datetime.date(2023, 1, 3),
       datetime.date(2023, 1, 4), datetime.date(2023, 1, 5),
       datetime.date(2023, 1, 6), datetime.date(2023, 1, 7),
       datetime.date(2023, 1, 8), datetime.date(2023, 1, 9),
       datetime.date(2023, 1, 10), datetime.date(2009, 1, 1),
       datetime.date(2023, 1, 11), datetime.date(2023, 1, 12),
       datetime.date(2023, 1, 13), datetime.date(2023, 2, 1),
       datetime.date(2023, 1, 14), datetime.date(2023, 1, 15),
       datetime.date(2023, 1, 16), datetime.date(2023, 1, 17),
       datetime.date(2023, 1, 18), datetime.date(2023, 1, 19),
       datetime.date(2023, 1, 20), datetime.date(2023, 1, 21),
       datetime.date(2023, 1, 22), datetime.date(2023, 1, 23),
       datetime.date(2023, 1, 24), datetime.date(2023, 1, 25),
       datetime.date(2023, 1, 26), datetime.date(2023, 1, 27),
       datetime.date(2023, 1, 28), datetime.date(2023, 1, 29),
   

Vamos a buscar quedarnos con la hora en formato entero

In [16]:
taxis['horario_inicio'].unique()

array([datetime.time(0, 26, 10), datetime.time(0, 51, 3),
       datetime.time(0, 35, 12), ..., datetime.time(5, 14, 58),
       datetime.time(2, 52, 53), datetime.time(4, 56, 34)], dtype=object)

In [17]:
taxis['horario_inicio'] = taxis['horario_inicio'].astype(str)

taxis['hora_inicio'] = taxis['horario_inicio'].str.split(':').str[0]

taxis['hora_inicio'] = taxis['hora_inicio'].astype(int)

print(taxis[['horario_inicio', 'hora_inicio']].head())


  horario_inicio  hora_inicio
0       00:26:10            0
1       00:51:03            0
2       00:35:12            0
3       00:13:14            0
4       00:33:04            0


In [18]:
taxis['hora_inicio'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7, 14,  8,  9, 10, 11, 12, 13, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [19]:
taxis['fecha_final'].unique()

array([datetime.date(2023, 1, 1), datetime.date(2023, 1, 2),
       datetime.date(2022, 12, 9), datetime.date(2023, 1, 3),
       datetime.date(2023, 1, 4), datetime.date(2023, 1, 5),
       datetime.date(2023, 1, 6), datetime.date(2023, 1, 7),
       datetime.date(2023, 1, 8), datetime.date(2023, 1, 9),
       datetime.date(2023, 1, 10), datetime.date(2023, 1, 11),
       datetime.date(2009, 1, 2), datetime.date(2023, 1, 12),
       datetime.date(2023, 1, 13), datetime.date(2023, 1, 14),
       datetime.date(2023, 2, 1), datetime.date(2023, 1, 15),
       datetime.date(2023, 1, 16), datetime.date(2023, 1, 17),
       datetime.date(2023, 1, 18), datetime.date(2023, 1, 19),
       datetime.date(2023, 1, 20), datetime.date(2023, 1, 21),
       datetime.date(2023, 1, 22), datetime.date(2023, 1, 23),
       datetime.date(2023, 1, 24), datetime.date(2023, 1, 25),
       datetime.date(2023, 1, 26), datetime.date(2023, 1, 27),
       datetime.date(2023, 1, 28), datetime.date(2023, 1, 29),
   

Tambien vamos a realizar lo mismo que hora_inico con horario final

In [20]:
taxis['horario_final'] = taxis['horario_final'].astype(str)

taxis['hora_final'] = taxis['horario_final'].str.split(':').str[0]

taxis['hora_final'] = taxis['hora_final'].astype(int)
print(taxis[['horario_final', 'hora_final']].head())


  horario_final  hora_final
0      00:37:11           0
1      00:57:49           0
2      00:41:32           0
3      00:19:03           0
4      00:39:02           0


Procedemos a eliminar las columas que no vamos a utilizar

In [21]:
taxis.head(1)

,passenger_count,trip_distance,total_amount,Servicio,Pickup_borough,fecha_inicio,horario_inicio,fecha_final,horario_final,hora_inicio,hora_final
0,1,2.58,24.18,Green,Manhattan,2023-01-01,00:26:10,2023-01-01,00:37:11,0,0


In [22]:
taxis = taxis.drop(columns=['horario_inicio', 'horario_final'])

In [23]:
taxis.head(1)

,passenger_count,trip_distance,total_amount,Servicio,Pickup_borough,fecha_inicio,fecha_final,hora_inicio,hora_final
0,1,2.58,24.18,Green,Manhattan,2023-01-01,2023-01-01,0,0


Vamos a extraer el los dias de la fecha. Arrancando con 0 el valor del dia lunes y 6 para domingo

In [24]:
def obtener_dia_semana(fecha):
    dia_semana = fecha.weekday()
    return dia_semana

taxis['dia_semana'] = taxis['fecha_inicio'].apply(obtener_dia_semana)

print(taxis[['fecha_inicio', 'dia_semana']])

        fecha_inicio  dia_semana
0         2023-01-01           6
1         2023-01-01           6
2         2023-01-01           6
3         2023-01-01           6
4         2023-01-01           6
...              ...         ...
1846358   2023-12-21           3
1846359   2023-12-13           2
1846360   2023-12-12           1
1846361   2023-12-14           3
1846362   2023-12-20           2

[1790230 rows x 2 columns]


In [25]:
taxis['dia_semana'].unique()

array([6, 4, 0, 1, 2, 3, 5])

In [26]:
taxis['dia_semana_viaje_final'] = taxis['fecha_final'].apply(obtener_dia_semana)

print(taxis[['fecha_final', 'dia_semana_viaje_final']])

        fecha_final  dia_semana_viaje_final
0        2023-01-01                       6
1        2023-01-01                       6
2        2023-01-01                       6
3        2023-01-01                       6
4        2023-01-01                       6
...             ...                     ...
1846358  2023-12-21                       3
1846359  2023-12-13                       2
1846360  2023-12-12                       1
1846361  2023-12-14                       3
1846362  2023-12-20                       2

[1790230 rows x 2 columns]


In [27]:
taxis['dia_semana_viaje_final'].unique()

array([6, 0, 4, 1, 2, 3, 5])

In [28]:
taxis = taxis.drop(columns=['fecha_inicio', 'fecha_final'])
taxis.head(1)

,passenger_count,trip_distance,total_amount,Servicio,Pickup_borough,hora_inicio,hora_final,dia_semana,dia_semana_viaje_final
0,1,2.58,24.18,Green,Manhattan,0,0,6,6


In [29]:
taxis.head(5)

,passenger_count,trip_distance,total_amount,Servicio,Pickup_borough,hora_inicio,hora_final,dia_semana,dia_semana_viaje_final
0,1,2.58,24.18,Green,Manhattan,0,0,6,6
1,1,1.81,15.84,Green,Manhattan,0,0,6,6
2,1,0.00,11.64,Green,Queens,0,0,6,6
3,1,1.30,10.20,Green,Manhattan,0,0,6,6
4,1,1.10,8.00,Green,Manhattan,0,0,6,6


In [30]:
missing_values = taxis.isnull().sum()

print("Valores nulos por columna:")
print(missing_values)

Valores nulos por columna:
passenger_count           0
trip_distance             0
total_amount              0
Servicio                  0
Pickup_borough            0
hora_inicio               0
hora_final                0
dia_semana                0
dia_semana_viaje_final    0
dtype: int64


Ahora vamos a cambiar los districtos y los servicios

In [31]:
taxis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1790230 entries, 0 to 1846362
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   passenger_count         int64  
 1   trip_distance           float64
 2   total_amount            float64
 3   Servicio                object 
 4   Pickup_borough          object 
 5   hora_inicio             int64  
 6   hora_final              int64  
 7   dia_semana              int64  
 8   dia_semana_viaje_final  int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 136.6+ MB


In [32]:
# Codificar las variables categóricas utilizando codificación ordinal
district_mapping = {'Manhattan': 0, 'Queens': 1, 'Brooklyn': 2, 'Staten Island': 3, 'EWR': 4}
service_mapping = {'Green': 0, 'Yellow': 1}

taxis['Pickup_borough'] = taxis['Pickup_borough'].map(district_mapping)
taxis['Servicio'] = taxis['Servicio'].map(service_mapping)

# Limitar los distritos a solo 5 y los tipos de taxi a 2
taxis['Pickup_borough'] = taxis['Pickup_borough'].astype('category')
taxis['Servicio'] = taxis['Servicio'].astype('category')
taxis.head(2)

,passenger_count,trip_distance,total_amount,Servicio,Pickup_borough,hora_inicio,hora_final,dia_semana,dia_semana_viaje_final
0,1,2.58,24.18,0,0,0,0,6,6
1,1,1.81,15.84,0,0,0,0,6,6


In [33]:
taxis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1790230 entries, 0 to 1846362
Data columns (total 9 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   passenger_count         int64   
 1   trip_distance           float64 
 2   total_amount            float64 
 3   Servicio                category
 4   Pickup_borough          category
 5   hora_inicio             int64   
 6   hora_final              int64   
 7   dia_semana              int64   
 8   dia_semana_viaje_final  int64   
dtypes: category(2), float64(2), int64(5)
memory usage: 112.7 MB


In [34]:
taxis['Servicio'].unique()

[0, 1]
Categories (2, int64): [0, 1]

In [35]:
taxis['Pickup_borough'].unique()

[0, 1, 2, 3, 4]
Categories (5, int64): [0, 1, 2, 3, 4]

VAMOS A DEFINIR LOS DATOS A UTILIZAR PARA IMPLEMENTAR EL MODELO

**Características:**

- Horario del día (hora_inicio)
- Día de la semana (dia_semana)
- Distrito de recogida (Pickup_borough)
- Tipo de servicio (Servicio)

**Variable objetivo:**

- Cantidad de pasajeros (passenger_count)

UNA VEZ QUE FINALIZAMOS CON LO QUE SERIAN TODAS LAS TRANSFORMACIONES PROCEDEMOS A REALIZAR EL MODELO DE MACHINE LEARNING

VAMOS A PROCEDER A SEPARAR LOS DATOS PARA ENTRENARLOS Y OTROS PARA PRUEBAS 

*DIVIDIMOS EL ARCHIVO*

Vamos a entrenar el 80% de los datos y el 20% vamos a dejar para prueba

In [36]:
taxis['demand'] = 1
demand_data = taxis.groupby(['hora_inicio', 'dia_semana', 'Pickup_borough', 'Servicio']).size().reset_index(name='demand')
taxis.head(4)

,passenger_count,trip_distance,total_amount,Servicio,Pickup_borough,hora_inicio,hora_final,dia_semana,dia_semana_viaje_final,demand
0,1,2.58,24.18,0,0,0,0,6,6,1
1,1,1.81,15.84,0,0,0,0,6,6,1
2,1,0.00,11.64,0,1,0,0,6,6,1
3,1,1.30,10.20,0,0,0,0,6,6,1


ENTRENAMIENTO DEL MODELO

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Codificar variables categóricas
demand_data['Servicio'] = demand_data['Servicio'].astype('category').cat.codes
demand_data['Pickup_borough'] = demand_data['Pickup_borough'].astype('category').cat.codes

# Selección de características y variable objetivo
features = demand_data[['hora_inicio', 'dia_semana', 'Pickup_borough', 'Servicio']]
target = demand_data['demand']

# División de los datos
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Entrenamiento del modelo
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predicciones
y_pred = model.predict(X_test)

# Evaluación del modelo
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f'RMSE: {rmse}')


RMSE: 246.92756905040565


In [38]:
def predict_demand(hora_inicio, dia_semana, pickup_borough, servicio):
    input_data = pd.DataFrame({
        'hora_inicio': [hora_inicio],
        'dia_semana': [dia_semana],
        'Pickup_borough': [pickup_borough],
        'Servicio': [servicio]
    })
    
   
    prediction = model.predict(input_data)
    return int(round(prediction[0]))

# Ejemplo de uso
hora_inicio = 6 ## van desde las 0hs a 23hs
dia_semana = 1  ## 'Lunes': 0, 'Martes': 1, 'Miercoles': 2, 'Jueves': 3, 'Viernes': 4, 'Sabado': 5, 'Domingo': 6
pickup_borough = 0  ##'Manhattan': 0, 'Queens': 1, 'Brooklyn': 2, 'Staten Island': 3, 'EWR': 4
servicio = 1  ## 'Green': 0, 'Yellow': 1

predicted_demand = predict_demand(hora_inicio, dia_semana, pickup_borough, servicio)
print(f"Predicted demand: {predicted_demand}")


Predicted demand: 2085


In [39]:
# Ejemplo de uso
hora_inicio = 20 ## van desde las 0hs a 23hs
dia_semana = 6  ## 'Lunes': 0, 'Martes': 1, 'Miercoles': 2, 'Jueves': 3, 'Viernes': 4, 'Sabado': 5, 'Domingo': 6
pickup_borough = 2  ##'Manhattan': 0, 'Queens': 1, 'Brooklyn': 2, 'Staten Island': 3, 'EWR': 4
servicio = 0  ## 'Green': 0, 'Yellow': 1

predicted_demand = predict_demand(hora_inicio, dia_semana, pickup_borough, servicio)
print(f"Predicted demand: {predicted_demand}")

Predicted demand: 783


In [40]:
print(predict_demand(10, 0, 0, 0))  # Horario, Dia de la semana, Manhattan, Yellow
print(predict_demand(23, 1, 1, 0))  # Horario, Dia de la semana, Queens, Yellow
print(predict_demand(6, 2, 2, 1))  # Horario, Dia de la semana, Brooklyn, Green
print(predict_demand(8, 3, 3, 1))  # Horario, Dia de la semana, Staten Island, Green
print(predict_demand(1, 1, 4, 0))  # Horario, Dia de la semana, EWR, Yellow

4066
973
46
0
1


Importar con jobLib

In [41]:
import joblib
joblib.dump(model, 'taxi_demand_model.joblib')

['taxi_demand_model.joblib']

Importar Con Pickle

In [42]:
import pickle

# Guardar el modelo entrenado en un archivo pickle
with open('taxi_demand_model.pkl', 'wb') as archivo:
    pickle.dump(model, archivo)

print("Modelo guardado en taxi_demand_model.pkl")


Modelo guardado en taxi_demand_model.pkl
